<a href="https://colab.research.google.com/github/IssacChew/interpolation/blob/master/Algo_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#To install osmnx
#conda config --prepend channels conda-forge
#conda create -n ox --strict-channel-priority osmnx
#conda activate ox
#Other libraries just use pip
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
import sys 
import pandas as pd
from geopy import distance
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import plotly.express as px
from plotly.offline import plot

ModuleNotFoundError: ignored

# Q1a) Plot Origin(UM) & Destination(Penang War Museum) 

In [ ]:
G_KL = ox.graph_from_point((3.123661, 101.612524),dist=10000,dist_type='network',network_type='drive')
G_Penang = ox.graph_from_point((5.288201, 100.275435),dist=5000, dist_type='bbox', network_type='drive')
m = folium.Map(location=ox.geocode("University of Malaya"))


#My Origin - darkpurple
origin = {"UM":ox.geocode("University of Malaya")}
folium.Marker(location=origin["UM"],
              icon=folium.Icon(color='darkpurple')).add_to(m)

#My Destination - darkpurple
dest = {"Museum":ox.geocode("Penang War Museum")}
folium.Marker(location=dest["Museum"],
              icon=folium.Icon(color='darkpurple')).add_to(m)


NameError: ignored

# Q1b) Plot Stops (Airports, Bus Stops, Grab Pick Up)

In [ ]:
#Airport - red
airports = {"Subang Airport":ox.geocode("Subang Airport"),"Penang Airport":ox.geocode("Penang Airport")}
for value in airports.values():
    folium.Marker(location=value,
              icon=folium.Icon(color='red')).add_to(m)
G_SJAir = ox.graph_from_point(airports["Subang Airport"],dist=5000, dist_type='bbox', network_type='drive')
G = nx.compose(G_SJAir,G_KL)
G = nx.compose(G,G_Penang)
G.add_edge(ox.get_nearest_node(G,airports["Subang Airport"]),ox.get_nearest_node(G,airports["Penang Airport"]))


#Bus - blue
bus_stops = {"KL Sentral":(3.134089, 101.686696),"One Utama": (3.15076, 101.615403),"TBS":(3.076056,101.711394),"Nearest Bus Stop To Museum":(5.278619, 100.271064)}
for value in bus_stops.values():
       folium.Marker(location=value,
              icon=folium.Icon(color='blue')).add_to(m)

#Grab Pick up - green
grab_pick_up = {"UM Sentral":(3.122332, 101.653279),"Taxi Subang Airport":(3.113430, 101.558535),
                 "Penang Airport Taxi Service":(5.302324, 100.274706)}
for value in grab_pick_up.values():
    folium.Marker(location=value,
              icon=folium.Icon(color='green')).add_to(m)
#Show Map
m

# Q1c) Options of Routes
## From UM to Kek Lok Si Temple
#### Option 1: UM Grab to Subang Airport ->  Flight to Penang Airport -> Taxi To Penang War Museum
#### Option 2: UM Grab to Subang Airport ->  Flight to Penang Airport -> Bus To Bus Stop  near Penang War Museum -> Walk 
#### Option 3: UM Grab to TBS - > Bus to Subang Airport ->  Flight to Penang Airport -> Taxi To Penang War Museum
#### Option 4: UM Grab to KL Sentral -> Bus to Subang Airport ->  Flight to Penang Airport -> Bus To Penang War Museum
#### Option 5: UM Grab to One Utama -> Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum

# Q2) Calculate the distances between Stops

In [ ]:
stops = {**origin,**airports,**bus_stops, **grab_pick_up,**dest}
penang = ["Penang Airport Taxi Service","Penang Airport","Museum","Nearest Bus Stop To Museum"]
distances_matrix = pd.DataFrame([[distance.distance(geocode1, geocode2).km 
                                  if (place1 in penang and place2 in penang) or (place1 not in penang and place2 not in penang) else 0
                                  for place1,geocode1 in stops.items()]  
                   for place2,geocode2 in stops.items()], columns=stops.keys(), index=stops.keys() )
distances_matrix.loc["Subang Airport","Penang Airport"] = distances_matrix.loc["Penang Airport","Subang Airport"] \
= distance.distance(airports["Subang Airport"], airports["Penang Airport"]).km 
paths = { place: [0,[]] for place in stops.keys()}
paths["UM"][1].append("UM")
distances_matrix


,UM,Subang Airport,Penang Airport,KL Sentral,One Utama,TBS,Nearest Bus Stop To Museum,UM Sentral,Taxi Subang Airport,Penang Airport Taxi Service,Museum
UM,0.000000,11.452104,0.000000,3.893395,5.256881,8.240130,0.000000,0.049126,10.612039,0.000000,0.000000
Subang Airport,11.452104,0.000000,278.279654,15.093738,7.469990,18.869055,0.000000,11.424248,2.191334,0.000000,0.000000
Penang Airport,0.000000,278.279654,0.000000,0.000000,0.000000,0.000000,1.874042,0.000000,0.000000,0.844944,2.528591
KL Sentral,3.893395,15.093738,0.000000,0.000000,8.136039,6.979739,0.000000,3.935397,14.427761,0.000000,0.000000
One Utama,5.256881,7.469990,0.000000,8.136039,0.000000,13.493932,0.000000,5.254162,7.549559,0.000000,0.000000
TBS,8.240130,18.869055,0.000000,6.979739,13.493932,0.000000,0.000000,8.241074,17.486900,0.000000,0.000000
Nearest Bus Stop To Museum,0.000000,0.000000,1.874042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.652292,2.001703
UM Sentral,0.049126,11.424248,0.000000,3.935397,5.254162,8.241074,0.000000,0.000000,10.577250,0.000000,0.000000
Taxi Subang Airport,10.612039,2.191334,0.000000,14.427761,7.549559,17.486900,0.000000,10.577250,0.000000,0.000000,0.000000
Penang Airport Taxi Service,0.000000,0.000000,0.844944,0.000000,0.000000,0.000000,2.652292,0.000000,0.000000,0.000000,2.814500


# Q3) Identify Shortest Path using Dijkstra Algorithm 

In [ ]:
class Graph(): 
   
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = pd.DataFrame([[0 for column in range(vertices)] for _ in range(vertices)])
        self.paths = { str(i): [0,[]] for i in range(vertices)}
   
    def minDistance(self, paths, sptSet): 
        min = sys.maxsize 
        for place in paths.keys(): 
            if paths[place][0] < min and sptSet[place] == False: 
                min = paths[place][0]
                min_place = place 
   
        return min_place 
   
    def dijkstra(self, src): 
   
        for i in paths.keys():
            paths[i][0] = sys.maxsize
        paths[src][0] = 0
        sptSet = {place:False for place in paths.keys()}
   
        for _ in range(self.V): 
   
            min_place = self.minDistance(paths, sptSet) 
   
            sptSet[min_place] = True
   
            for place in paths.keys(): 
                if self.graph.loc[min_place,place] > 0 and sptSet[place] == False and \
            paths[place][0] > paths[min_place][0] + self.graph.loc[min_place,place]: 
                     paths[place][0] = paths[min_place][0] + self.graph.loc[min_place,place] 
                     paths[place][1] = paths[min_place][1] + [place]
   
        print(paths) 

# Driver program 
g = Graph(len(stops)) 
g.graph = distances_matrix
g.paths = paths
g.dijkstra("UM") 

{'UM': [0, ['UM']], 'Subang Airport': [11.452103554600093, ['UM', 'Subang Airport']], 'Penang Airport': [289.73175730845236, ['UM', 'Subang Airport', 'Penang Airport']], 'KL Sentral': [3.893394904485259, ['UM', 'KL Sentral']], 'One Utama': [5.256881255967034, ['UM', 'One Utama']], 'TBS': [8.240130385259599, ['UM', 'TBS']], 'Nearest Bus Stop To Museum': [291.6057993677916, ['UM', 'Subang Airport', 'Penang Airport', 'Nearest Bus Stop To Museum']], 'UM Sentral': [0.04912615914221051, ['UM', 'UM Sentral']], 'Taxi Subang Airport': [10.612038786503513, ['UM', 'Taxi Subang Airport']], 'Penang Airport Taxi Service': [290.57670145429074, ['UM', 'Subang Airport', 'Penang Airport', 'Penang Airport Taxi Service']], 'Museum': [292.26034814154235, ['UM', 'Subang Airport', 'Penang Airport', 'Museum']]}


# Q4) Plot Shortest Route

In [ ]:
#G_KL.add_edge(ox.get_nearest_node(G_Penang,airports[1]),ox.get_nearest_node(G_KL,airports[0]))
#G = nx.compose(G_KL,G_Penang)
#Shortest Path
                         
for place in zip(g.paths["Museum"][1],g.paths["Museum"][1][1:]):
    print(place)
    if place is not None:
        try:
            route = nx.shortest_path(G, 
                                 ox.get_nearest_node(G,stops[place[0]]),
                                 ox.get_nearest_node(G,stops[place[1]]),
                                 weight='length')
            ox.plot_route_folium(G, route, route_color='green',route_map=m)
        except:
            continue
route = [ox.get_nearest_node(G,stops["Subang Airport"]),ox.get_nearest_node(G,stops["Penang Airport"])]
ox.plot_route_folium(G, route, route_color='green',route_map=m)

#Show Map
m

('UM', 'Subang Airport')
('Subang Airport', 'Penang Airport')
('Penang Airport', 'Museum')


# Q5- Scrap an article


In [ ]:
def buildWordsFrequency(url):
    """
    params: url
    return: DataFrame(Column= ["Frequency"], Index=["Word"])
    """
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    body = [words.getText().lower().split() for words in soup.find_all(['h1', 'h2','h3','h4','h5','h6',"p"])]
    #words = [ re.sub("^[^a-z]+|[^a-z]+$","",word) for sentence in body for word in sentence]
    words = [ re.sub("[^a-z]+","",word) for sentence in body for word in sentence]
    words = pd.DataFrame(words,columns=["Word"])
    words['Word'].replace('', np.nan, inplace=True)
    words.dropna(subset=['Word'], inplace=True)
    words['Frequency'] = words.groupby(['Word'])['Word'].transform('count')
    words.drop_duplicates(subset=["Word"],inplace=True)
    words.set_index('Word', inplace=True)
    words.sort_values("Frequency",ascending=False, inplace=True)
    return words 

grab_url = "https://www.tripadvisor.com.my/ShowTopic-g298570-i7008-k12946278-Grab_Uber-Kuala_Lumpur_Wilayah_Persekutuan.html"
taxi_url = "https://www.tripadvisor.com.my/ShowTopic-g298570-i7008-k5961236-How_s_taxi_service_in_KL-Kuala_Lumpur_Wilayah_Persekutuan.html"
bus_url = "https://www.tripadvisor.com.my/ShowTopic-g293951-i7006-k7552812-Malaysia_Express_Bus_Review_Continuous_Updates-Malaysia.html"
airline_url = "https://www.airlinequality.com/airline-reviews/malaysia-airlines/"
words = {"Grab": buildWordsFrequency(grab_url), "Taxi":buildWordsFrequency(taxi_url), "Bus": buildWordsFrequency(bus_url),"Airline": buildWordsFrequency(airline_url)}

In [ ]:
#Scrap Stop Words
URL = 'https://www.ranks.nl/stopwords'
page = requests.get(URL,verify=False)
soup = BeautifulSoup(page.content, 'html.parser')
row = soup.find('tr')
cols = row.find_all('td')
cols = [re.split("<[^<>]*>",str(ele)) for ele in cols]
stop_words = [ stop_word.lower() for stop_words in cols for stop_word in stop_words if stop_word]

C:\Users\BrandonTan\anaconda3\envs\ox\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'www.ranks.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



# Tries

In [ ]:
class TrieNode: 
        # Trie node class 
        def __init__(self): 
            self.children = [None]*30

            # isEndOfWord is True if node represent the end of the word 
            self.isEndOfWord = False

In [ ]:
class Trie: 

    # Trie data structure class 
    def __init__(self): 
        self.root = self.getNode() 
  
    def getNode(self): 
      
        # Returns new trie node (initialized to NULLs) 
        return TrieNode() 
  
    def _charToIndex(self,ch): 
          
        # private helper function 
        # Converts key current character into index 
        # use only 'a' through 'z' and lower case 
      
        if ch=="'":
            return 26
        elif ch == '-':
            return 27
        elif ch == "*":
            return 28
        elif ch == "—":
            return 29
        return ord(ch)-ord('a') 
  
  
    def insert(self,key): 
    
        
        # If not present, inserts key into trie 
        # If the key is prefix of trie node,  
        # just marks leaf node 
        pCrawl = self.root 
        length = len(key) 
        for level in range(length): 
            index = self._charToIndex(key[level]) 
           
            # if current character is not present 
            if not pCrawl.children[index]: 
                pCrawl.children[index] = self.getNode() 
            pCrawl = pCrawl.children[index] 
  
        # mark last node as leaf 
        pCrawl.isEndOfWord = True
  
    def search(self, key): 
          
        # Search key in the trie 
        # Returns true if key presents  
        # in trie, else false 
        pCrawl = self.root 
        length = len(key) 
        for level in range(length): 
            index = self._charToIndex(key[level]) 
            
            if not pCrawl.children[index]: 
                return False
            pCrawl = pCrawl.children[index] 
  
        return pCrawl != None and pCrawl.isEndOfWord 



In [ ]:
#Trie String Matcher - Stop Words
stop_words_trie = Trie()
for word in stop_words:
    stop_words_trie.insert(word)
for transport in words.keys():
    words[transport]["isStopWord"] = np.vectorize(stop_words_trie.search)(words[transport].index)

# Q6 - Ployly - Histogram of words

In [ ]:
for transport in words.keys():
    fig = px.bar(words[transport], x=words[transport].index, y='Frequency',color="isStopWord")
    fig.update_layout(title_text=transport)
    fig.show()

NameError: ignored

# Q7 - Trie String Matcher - Positive & Negative

In [ ]:
def build_trie(file):
    words = np.genfromtxt(file, delimiter=',',dtype=str, encoding="utf-8",autostrip=True)
    trie = Trie()
    words = np.char.lower(words)
    for word in words:
        if(len(word.split(" "))==1):
            if re.search("[^a-z'-]+",word)==None:
                trie.insert(word)  
    return trie
def search_wrapper(posTrie,negTrie,value):
    if posTrie.search(value) == True:
        return "Positive"
    elif negTrie.search(value) == True:
        return "Negative"
    else:
        return "Neutral"
    
pos_words_trie = build_trie(r'C:\Users\BrandonTan\Downloads\SEM 4\Algorithm & Design\Positive Words.csv')
neg_words_trie = build_trie(r'C:\Users\BrandonTan\Downloads\SEM 4\Algorithm & Design\Negative Words.csv')
pos_neg_words={}
for transport in words.keys():
    words_transport = words[transport]
    words_transport["Sentiment"] = np.vectorize(search_wrapper)(pos_words_trie,neg_words_trie,words_transport.index)
    pos_neg_words.update({transport:words_transport[(words_transport["Sentiment"]!="Neutral") & (words_transport["isStopWord"]==False)]})
   

# Q8 - Ployly - Histogram of Positive & Negative Words

In [ ]:
for transport in words.keys():
    pos_neg_words_transport = pos_neg_words[transport]
    fig = px.bar(pos_neg_words_transport, x=pos_neg_words_transport.index, y='Frequency',color="Sentiment")
    fig.update_layout(title_text=transport)
    fig.show()

# Q9a) Determine the nature of Article (Neg,Pos,Neutral)

In [ ]:
percent_of_pos = {}
for transport in pos_neg_words.keys():
    count = pos_neg_words[transport].groupby("Sentiment") ["Frequency"].sum()
    if count["Positive"] >= count["Negative"]:
        print(transport+": Positive")
    else:
        print(transport+"Negative")
    percent_of_pos.update({transport:count["Positive"]/(count["Positive"]+count["Negative"])})
    print(" Positive: "+str(count["Positive"])+" Negative: "+str(count["Negative"]))
    #Percentage of Postive: = count["Positive"]/(count["Positive"]+count["Negative"]
    print(" Percentage of Postive: "+str(percent_of_pos[transport]))

Grab: Positive
 Positive: 45 Negative: 7
 Percentage of Postive: 0.8653846153846154
Taxi: Positive
 Positive: 35 Negative: 9
 Percentage of Postive: 0.7954545454545454
Bus: Positive
 Positive: 45 Negative: 15
 Percentage of Postive: 0.75
Airline: Positive
 Positive: 21 Negative: 1
 Percentage of Postive: 0.9545454545454546


# Q9b) Accuracy of Determination of Articles Above
### 1) It is not accurate because:
####     a) I had analysed 1 article only for each transportation.
####     b) bias exists when I took an article with a better review.

# Q9c) Conclusion
### Grab: Positive
### Taxi: Positive
### Bus: Positive
### Airline:Positive

# Q10a) Calculate Distance for All Options

In [ ]:
options = pd.DataFrame([0]*5, columns= ["Distance"],index=[ "Option "+str(i+1) for i in range(5)])

#Option 1: UM Grab to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[0]+=distances_matrix.loc["UM","Subang Airport"]+ distances_matrix.loc["Subang Airport","Penang Airport"]+\
distances_matrix.loc["Subang Airport","Museum"]

#Option 2: UM Grab to Subang Airport -> Flight to Penang Airport -> Bus To Bus Stop near Penang War Museum -> Taxi To Penang War Museum
options.iloc[1]+=distances_matrix.loc["UM","Subang Airport"]+ distances_matrix.loc["Subang Airport","Penang Airport"]+\
distances_matrix.loc["Penang Airport","Nearest Bus Stop To Museum"]+distances_matrix.loc["Nearest Bus Stop To Museum","Museum"]

#Option 3: UM Grab to TBS - > Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[2]+=distances_matrix.loc["UM","TBS"]+ distances_matrix.loc["TBS","Subang Airport"]+\
distances_matrix.loc["Subang Airport","Penang Airport"]+ distances_matrix.loc["Penang Airport","Museum"]

#Option 4: UM Grab to KL Sentral -> Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[3]+=distances_matrix.loc["UM","KL Sentral"]+ distances_matrix.loc["KL Sentral","Subang Airport"]+\
distances_matrix.loc["Subang Airport","Penang Airport"]+ distances_matrix.loc["Penang Airport","Museum"]

#Option 5: UM Grab to One Utama -> Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[4]+=distances_matrix.loc["UM","One Utama"]+ distances_matrix.loc["One Utama","Subang Airport"]+\
distances_matrix.loc["Subang Airport","Penang Airport"]+distances_matrix.loc["Penang Airport","Museum"]

# Q10b) Calculate Sentiment for Each Transport for All Options

In [ ]:
options["Sentiment Score"] = 0

#Option 1: UM Grab to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[0,1]+= percent_of_pos["Grab"] +percent_of_pos["Airline"]+percent_of_pos["Taxi"]

#Option 2: UM Grab to Subang Airport -> Flight to Penang Airport -> Bus To Bus Stop near Penang War Museum -> Taxi To Penang War Museum
options.iloc[1,1]+= percent_of_pos["Grab"] +percent_of_pos["Airline"]+percent_of_pos["Bus"]+percent_of_pos["Taxi"]

#Option 3: UM Grab to TBS - > Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[2,1]+= percent_of_pos["Grab"] +percent_of_pos["Bus"]+percent_of_pos["Airline"]+percent_of_pos["Taxi"]

#Option 4: UM Grab to KL Sentral -> Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[3,1]+= percent_of_pos["Grab"] +percent_of_pos["Bus"] +percent_of_pos["Airline"]+percent_of_pos["Taxi"]

#Option 5: UM Grab to One Utama -> Bus to Subang Airport -> Flight to Penang Airport -> Taxi To Penang War Museum
options.iloc[4,1]+= percent_of_pos["Grab"] +percent_of_pos["Bus"] +percent_of_pos["Airline"]+percent_of_pos["Taxi"]


# Q10c) Summary

In [ ]:
options["Total Score"] = options["Distance"] - options["Sentiment Score"]
options

,Distance,Sentiment Score,Total Score
Option 1,289.731757,2.615385,287.116373
Option 2,293.607502,3.365385,290.242117
Option 3,307.917430,3.365385,304.552045
Option 4,299.795377,3.365385,296.429993
Option 5,293.535116,3.365385,290.169731
